# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [14]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
# Import API key
from api_keys import geoapify_key
import warnings
warnings.filterwarnings("ignore")

In [15]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,la'ie,21.6477,-157.9253,27.66,75,47,5.09,US,1737856058
1,1,invercargill,-46.4000,168.3500,16.12,81,100,4.47,NZ,1737856060
2,2,amga,60.8953,131.9608,-26.17,97,100,1.66,RU,1737856061
3,3,piacabucu,-10.4056,-36.4344,26.08,80,15,4.37,BR,1737856062
4,4,puerto natales,-51.7236,-72.4875,11.14,82,100,1.27,CL,1737856063


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
# Configure the map plot
map_plt_1=city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 600,
    size = "Humidity",
    scale = 0.9,
    color = "City"

)
#Display the map
map_plt_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
condition1= city_data_df[(city_data_df["Max Temp"]>21) & (city_data_df["Max Temp"]<27)]
condition2=condition1[condition1["Wind Speed"]<4.5]
narrow_data=condition2[condition2["Cloudiness"]==0]

# Drop any rows with null values
narrow_data_clean=narrow_data.dropna()

# Display sample data
narrow_data_clean

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,cabo san lucas,22.8909,-109.9124,21.53,77,0,2.57,MX,1737855683
43,43,port elizabeth,-33.9180,25.5701,23.51,87,0,2.57,ZA,1737856110
81,81,al hawatah,13.4167,34.6333,22.04,32,0,4.13,SD,1737856158
158,158,cidreira,-30.1811,-50.2056,23.73,93,0,1.99,BR,1737856257
186,186,graneros,-34.0667,-70.7333,21.91,33,0,0.28,CL,1737856296
307,307,maturin,9.7500,-63.1767,22.77,88,0,2.76,VE,1737856447
313,313,coahuayana de hidalgo,18.7000,-103.6583,22.66,91,0,0.46,MX,1737856455
338,338,brisas de zicatela,15.8369,-97.0419,24.95,64,0,2.44,MX,1737856487
359,359,upington,-28.4478,21.2561,26.21,36,0,2.57,ZA,1737856515
363,363,mar del plata,-38.0023,-57.5575,21.02,75,0,4.12,AR,1737856278


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=city_data_df[["City","Country","Lat","Lng","Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,la'ie,US,21.6477,-157.9253,75,
1,invercargill,NZ,-46.4000,168.3500,81,
2,amga,RU,60.8953,131.9608,97,
3,piacabucu,BR,-10.4056,-36.4344,80,
4,puerto natales,CL,-51.7236,-72.4875,82,
...,...,...,...,...,...,...
542,el granada,US,37.5027,-122.4694,63,
543,kudahuvadhoo,MV,2.6708,72.8944,74,
544,letlhakeng,BW,-24.0944,25.0298,56,
545,elassona,GR,39.8947,22.1886,92,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories": categories,
    "apiKey": geoapify_key,
    "limit": 30
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    Latitude = row['Lat']
    Longitude = row['Lng']
    
    # Add the current city's latitude and longitude to the params dictionary
   
    params["bias"] = f"proximity:{Longitude},{Latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        # Convert the API response to JSON format
        name_address = response.json()

        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_name = name_address["features"][0]["properties"]["name"]
            hotel_df.loc[index, "Hotel Name"] = hotel_name
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found"
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    else:
        # Handle the error if the API call fails
        hotel_df.loc[index, "Hotel Name"] = "API request failed"
        print(f"Error for {row['City']}: {response.status_code}")

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
east london - nearest hotel: No hotel found
kingston - nearest hotel: Aphrodite Hotel
nizhneangarsk - nearest hotel: Байкал-Сервис
papatowai - nearest hotel: Catlins Woodstock Lodge & Camp
san antonio de pale - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
rio grande - nearest hotel: Hotel Vila Moura Executivo
aasiaat - nearest hotel: SØMA
hamilton - nearest hotel: North Vista Manor
youhao - nearest hotel: No hotel found
kutum - nearest hotel: Noor Aleman
tabora - nearest hotel: Alpho hotel
invercargill - nearest hotel: The Grand
kodiak - nearest hotel: No hotel found
senador jose porfirio - nearest hotel: Pousada Brasil
udachny - nearest hotel: Вилюй
adamstown - nearest hotel: Chez Bianca & Benoit
badger - nearest hotel: Clarion Hotel & Suites
bethel - nearest hotel: Hampton Inn Danbury
nikolayevsk-on-amure - nearest hotel: Север
iqaluit - nearest hotel: Frobisher Inn
aykhal - nearest hotel: Уют
magadan - nearest hotel: Гостиница «ВМ-Це

,City,Country,Lat,Lng,Humidity,Hotel Name
0,east london,ZA,-33.0153,27.9116,88,No hotel found
1,kingston,JM,17.9970,-76.7936,70,Aphrodite Hotel
2,nizhneangarsk,RU,55.7833,109.5500,86,Байкал-Сервис
3,papatowai,NZ,-46.5619,169.4708,97,Catlins Woodstock Lodge & Camp
4,san antonio de pale,GQ,-1.4014,5.6325,81,No hotel found
...,...,...,...,...,...,...
536,olyokminsk,RU,60.3736,120.4264,96,Колос
537,zalingei,SD,12.9096,23.4741,17,Princess
538,mitsamiouli,KM,-11.3847,43.2844,85,Hotel le Paradis
539,isangel,VU,-19.5500,169.2667,78,Tanna Lodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
# Configure the map plot
map_plt_2=hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols =['City','Country','Hotel Name'],
    size = "Humidity",
    scale = 1,
    color = "City"

)
#Display the map
map_plt_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)